# Microscopie hyperspectrale

Quentin Douzery  
Alexia Ghozland

## 1. Imports

### 1.1. Packages

In [62]:
import h5py
import numpy as np
import os
from joblib import Parallel, delayed
import math

In [63]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 1.2. Images

In [64]:
#Dictionnaire qui contient les fichiers des images originales
#path_dir = "/content/gdrive/My Drive/Hyperspectral" #drive directory
path_dir = "/content/gdrive/My Drive/Hyperspectral/Images-hyperspectrales" 
#path_dir = "/home/douzery/Bureau" #local directory
raw_images_names = {"j5_1_740nm":os.path.join(path_dir, "j5-740nm/hdf5-images/j5_1_740nm.h5")}
#raw_images_names = {"j5_1_740nm":os.path.join(path_dir, "Test-hdf5/j5_1_740nm.h5")}

In [65]:
#Dictionnaire qui contient les fichiers des images après segmentation
#segmented_images_names = {"j5_1_740nm-s":os.path.join(path_dir, "Test-hdf5/j5_1_740nm-segmented.h5")}
segmented_images_names = {"j5_1_740nm-s":os.path.join(path_dir, "j5-740nm/segmented-images/j5_1_740nm_Simple-Segmentation.h5")}

In [66]:
#Dictionnaire qui contient les images originales (format hdf5)
aux_raw_images = {}
for name, path in raw_images_names.items():
    aux_raw_images[name] = h5py.File(path, 'r')

In [67]:
#Dictionnaire qui contient les images après segmentation (format hdf5)
aux_segmented_images = {}
for name, path in segmented_images_names.items():
    aux_segmented_images[name] = h5py.File(path, 'r')

In [68]:
#Dictionnaire qui contient les images originales (utilisables, tableaux Python)
raw_images = {}
for name, img in aux_raw_images.items():
  raw_images[name] = np.reshape(aux_raw_images[name]['data'], (12,512,512,32))

In [69]:
#Dictionnaire qui contient les images après segmentation (utilisables, tableau Python)
segmented_images = {}
for name, img in aux_segmented_images.items():
  segmented_images[name] = np.reshape(aux_segmented_images[name]['exported_data'], (12,512,512))

## 2. Caractéristiques des images

In [70]:
print("Dimensions de l'image originale (z, y, x, c) :", raw_images["j5_1_740nm"].shape)
print("Dimensions de l'image segmentée (z, y, x) :", segmented_images["j5_1_740nm-s"].shape)

Dimensions de l'image originale (z, y, x, c) : (12, 512, 512, 32)
Dimensions de l'image segmentée (z, y, x) : (12, 512, 512)


## 3. Aperçu des images

In [71]:
print("Valeurs des 32 canaux du premier voxel de l'image originale :", raw_images["j5_1_740nm"][0][0][0][:], "\n")
print("Valeur du premier voxel de l'image segmentée (1 = muscle // 2 = non muscle) :", segmented_images["j5_1_740nm-s"][0][0][0])

Valeurs des 32 canaux du premier voxel de l'image originale : [173 236 207 191  34 160  45 193 187 247  64  93 312  84 264 204 140 224
 125 180  95 146 130 183 179 120   2 119 261 158 177 119] 

Valeur du premier voxel de l'image segmentée (1 = muscle // 2 = non muscle) : 2


## 4. Modification des images segmentées

In [72]:
#Création d'un "filtre" pour isoler les valeurs des voxels correspondant à du muscle
filtre_muscle = segmented_images["j5_1_740nm-s"] - 2*np.ones((12,512,512))
filtre_muscle = np.abs(filtre_muscle)

In [73]:
#Création d'un "filtre" pour isoler les valeurs des voxels correspondant à ce qui n'est pas du muscle
filtre_autre = segmented_images["j5_1_740nm-s"] - np.ones((12,512,512))
filtre_autre = np.abs(filtre_autre)

## 5. Spectres moyens des images

In [116]:
test = filtre_muscle*raw_images["j5_1_740nm"][:,:,:,0]
test2 = filtre_autre*raw_images["j5_1_740nm"][:,:,:,0]

In [120]:
print(np.sum(test)+np.sum(test2))
print(np.sum(raw_images["j5_1_740nm"][:,:,:,0]))

2038928942.0
2038928942


In [128]:
unique, counts = np.unique(filtre_muscle, return_counts=True)
dict(zip(unique, counts))
print(counts[0])
print(counts[1])

2493321
652407


In [121]:
unique, counts = np.unique(filtre_muscle, return_counts=True)
h = filtre_muscle.count(0)
f = 512*512*12
print(g+h)
print(f)

AttributeError: ignored

In [101]:
test = np.copy(raw_images["j5_1_740nm"])

In [103]:
test[:][:][:][0].shape

(512, 512, 32)

In [110]:
t = np.zeros((1,2,3))
t[0][0][0] = 1
t[0][1][0] = 2
t

array([[[1., 0., 0.],
        [2., 0., 0.]]])

In [111]:
t2 = np.reshape(t, (2,3))
t2[:,0]

array([1., 2.])

In [51]:
raw_images["j5_1_740nm"][0].shape

(512, 512, 32)

In [112]:
raw_images["j5_1_740nm"][:,:,:,0].shape

(12, 512, 512)

In [98]:
filter = np.zeros((1,3,2,2))
filter[0][0][0][0]=1
filter[0][0][0][1]=1
print(filter)
filter2 = filter.reshape(3,2,2)
print(filter2)
print(filter.shape)
print(filter2[0][:][:].shape)

[[[[1. 1.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
[[[1. 1.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
(1, 3, 2, 2)
(2, 2)


In [22]:
t = np.ones((2,2,2,2))
filter = np.zeros((1,2,2,2))
filter[0][0][0]=1
filter[0][0][1]=1
print(t)
print(filter)
print(filter*t)

[[[[1. 1.]
   [1. 1.]]

  [[1. 1.]
   [1. 1.]]]


 [[[1. 1.]
   [1. 1.]]

  [[1. 1.]
   [1. 1.]]]]
[[[1. 1.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
[[[[1. 1.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[1. 1.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]


In [ ]:
from tqdm import tqdm

In [ ]:
print(dset.shape[2])

512


In [ ]:
MEAN_CHANNELS = np.zeros(32)

In [ ]:
def test(dset) :
  if (dset[0][z][y][x][0]==1)
#MEAN_CHANNELS[c] += dset[0][z][y][x][c]
#def sqrt_func(i, j):
    #time.sleep(1)
    #return math.sqrt(i**j)

#Parallel(n_jobs=2)(delayed(sqrt_func)(i, j) for i in range(5) for j in range(2))


In [ ]:

for c in range(dset.shape[4]):
  for z in range(dset.shape[1]):
    for y in range(dset.shape[2]):
      for x in range(dset.shape[3]):
        if (dset2[0][z][y][x][0] == 1):
          pass

0


KeyboardInterrupt: ignored

In [ ]:
channels = np.arange(0,32,1)
channels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [ ]:
Muscle = pd.DataFrame(test, columns=["Channel number", "Mean channel value"])

In [ ]:
#if (dset2[0][z][y][x][0] == 1):
#print(x)
#MEAN_CHANNELS[c] += dset[0][z][y][x][c]

## Test dataframe

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(dset)

ValueError: ignored

In [ ]:
t = np.ones((3,3,3))
filter = np.zeros((1,3,3,1))
filter[0][0][0][0]=1
filter[0][0][1][0]=4
filter[0][2][0][0]=1
filter2 = np.reshape(filter, (3,3))
print(t)
print(filter)
print(filter2)
print(filter[0][0][1][0])
print(filter2[0][1])

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
[[[[1.]
   [4.]
   [0.]]

  [[0.]
   [0.]
   [0.]]

  [[1.]
   [0.]
   [0.]]]]
[[1. 4. 0.]
 [0. 0. 0.]
 [1. 0. 0.]]
4.0
4.0


In [ ]:
T = [[-1,0],[-2,1]]

In [ ]:
absT = np.abs(T)

In [ ]:
print(T)
print(absT)

[[-1, 0], [-2, 1]]
[[1 0]
 [2 1]]


In [ ]:
A = [[1,2],[3,4]]
B = [[1,0],[0,1]]
C = np.multiply(A,B)

In [ ]:
print(C)

[[1 0]
 [0 4]]
